In [ ]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

from models.ReinforcementLearning.DeepQ_TicTacToe_v2.DeepQAgent import DeepQAgent
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from models.ReinforcementLearning.Utils import (
    train_agent,
    test_agent,
    set_seed
)

SEED = 100
set_seed(SEED)

# DeepQ parameters
BATCH_SIZE     = 128
NUM_EPISODES   = 1000 if torch.cuda.is_available() else 100
STATE_SPACE    = 9
ACTION_SPACE   = 9
HIDDEN_SIZE    = 128
EPSILON        = 1.0
GAMMA          = 0.99
LEARNING_RATE  = 0.001
DROPOUT        = 0.25
TRAIN_START    = 1500
NEGATIVE_SLOPE = 0.01
COMPUTER_LEVEL = OPPONENT_LEVEL.NAIVE

# save path
MODEL_PATH = "../../../../trained_models/ReinforcementLearning/TicTacToeV2"
MODEL_NAME = ""

BASELINE = "TicTacToev2-Baseline-Untrained"
NAIVE_2K = "TicTacToev2-NAIVE-2K"
NAIVE_4K = "TicTacToev2-NAIVE-4K"
NAIVE_6K = "TicTacToev2-NAIVE-6K"
NAIVE_8K = "TicTacToev2-NAIVE-8K"
NAIVE_10K = "TicTacToev2-NAIVE-10K"
AGENT_1K = "TicTacToev2-AGENT-1K"
AGENT_2K = "TicTacToev2-AGENT-2K"
AGENT_3K = "TicTacToev2-AGENT-3K"
AGENT_4K = "TicTacToev2-AGENT-4K"


def get_full_model_path(agent_name: str = None):
  if agent_name is None:
    return os.path.join(MODEL_PATH, MODEL_NAME + ".pt")
  return os.path.join(MODEL_PATH, agent_name + ".pt")

def supply_model(load_if_exists: bool = True, agent_name: str = None):
  
  agent = DeepQAgent(
      device         = DEVICE,
      epsilon        = EPSILON, 
      gamma          = GAMMA,
      state_space    = STATE_SPACE, 
      action_space   = ACTION_SPACE, 
      hidden_size    = HIDDEN_SIZE,
      dropout        = DROPOUT,
      train_start    = TRAIN_START,
      batch_size     = BATCH_SIZE,
      negative_slope = NEGATIVE_SLOPE
  )

  full_model_path = get_full_model_path(agent_name)
  if load_if_exists and os.path.exists(full_model_path):
    print("Loading Model Parameters...")
    agent.load_model(filepath=full_model_path)
  
  optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)
  criterion = torch.nn.SmoothL1Loss() # Huber Loss
  
  return agent, optimizer, criterion

def compare_to_naive(agent_name: str, num_episodes: int = 25000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_optimal(agent_name: str, num_episodes: int = 100):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.OPTIMAL, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_model(agent_name: str, model_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _enemy, _, _ = supply_model(load_if_exists=True, agent_name=model_name)
  _environment = TicTacToeGame(DEVICE, _enemy, OPPONENT_LEVEL.AGENT, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)


The NAIVE_2K and NAIVE_4K models seem to perform well against the the Naive opponent, and better than the other NAIVE models against the Optimal Opponent.

In [ ]:
compare_to_model(NAIVE_2K, BASELINE)

In [ ]:
compare_to_model(NAIVE_4K, BASELINE)

The NAIVE_2K model performs better against the BASELINE model than the NAIVE_4K model. It also performs better against the NAIVE opponent (see naive-train.ipynb). We'll focus on this model for agent training.

In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=NAIVE_2K)
enemy, _, _ = supply_model(load_if_exists=True, agent_name=NAIVE_2K)
environment = TicTacToeGame(DEVICE, enemy, OPPONENT_LEVEL.AGENT)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = AGENT_1K
)

In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=AGENT_1K)
enemy, _, _ = supply_model(load_if_exists=True, agent_name=AGENT_1K)
environment = TicTacToeGame(DEVICE, enemy, OPPONENT_LEVEL.AGENT)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = AGENT_2K
)

In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=AGENT_2K)
enemy, _, _ = supply_model(load_if_exists=True, agent_name=AGENT_2K)
environment = TicTacToeGame(DEVICE, enemy, OPPONENT_LEVEL.AGENT)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = AGENT_3K
)

In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=AGENT_3K)
enemy, _, _ = supply_model(load_if_exists=True, agent_name=AGENT_3K)
environment = TicTacToeGame(DEVICE, enemy, OPPONENT_LEVEL.AGENT)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = AGENT_4K
)

##### Comparing the AGENT models to a Naive opponent.

In [ ]:
compare_to_naive(BASELINE)

In [ ]:
compare_to_naive(NAIVE_2K)

In [ ]:
compare_to_naive(AGENT_1K)

In [ ]:
compare_to_naive(AGENT_2K)

In [ ]:
compare_to_naive(AGENT_3K)

In [ ]:
compare_to_naive(AGENT_4K)

The 2K, 3K, and 4K, AGENT models perform worse than BASELINE...

##### Comparing the AGENT models to a NAIVE_2K.

In [ ]:
compare_to_model(NAIVE_2K, NAIVE_2K)

In [ ]:
compare_to_model(AGENT_1K, NAIVE_2K)

In [ ]:
compare_to_model(AGENT_2K, NAIVE_2K)

In [ ]:
compare_to_model(AGENT_3K, NAIVE_2K)

In [ ]:
compare_to_model(AGENT_4K, NAIVE_2K)

##### Comparing AGENT models to Optimal opponent.

In [ ]:
compare_to_optimal(NAIVE_2K)

In [ ]:
compare_to_optimal(AGENT_1K)

In [ ]:
compare_to_optimal(AGENT_2K)

In [ ]:
compare_to_optimal(AGENT_3K)

In [ ]:
compare_to_optimal(AGENT_4K)